<a href="https://colab.research.google.com/github/dieko95/blackouts-C4V/blob/master/twitter_pretagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated Tagging

## Accessing Data

In [0]:
!pip install --upgrade -q gspread

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Training Set - #SinLuz Country Classifier').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

In [0]:
# Convert to a DataFrame and render.
import pandas as pd
# pd.DataFrame.from_records(rows)
tagOriginalDf = pd.DataFrame(rows)

tagOriginalDf.columns = tagOriginalDf.iloc[0,:]
tagOriginalDf = tagOriginalDf.drop(index = 0)

In [0]:
diegoTags = tagOriginalDf.iloc[6001:8500,:].copy()

diegoOriginalTag = diegoTags[['concat_text_user_description', 'label_country', 'label_state', 'label_type']].iloc[:271,:].copy()
diegoToTag = diegoTags[['concat_text_user_description', 'label_country', 'label_state', 'label_type']].iloc[271:,:].copy()

## Cleaning Text

In [0]:
def cleaner(df,text_col):
  # to lower

  df[text_col] = df[text_col].str.lower()

  # Convert common spanish accents

  df[text_col] = df[text_col].str.replace("ú", "u")
  df[text_col] = df[text_col].str.replace("ù", "u")
  df[text_col] = df[text_col].str.replace("ü", "u")
  df[text_col] = df[text_col].str.replace("ó", "o")
  df[text_col] = df[text_col].str.replace("ò", "o")
  df[text_col] = df[text_col].str.replace("í", "i")
  df[text_col] = df[text_col].str.replace("ì", "i")
  df[text_col] = df[text_col].str.replace("é", "e")
  df[text_col] = df[text_col].str.replace("è", "e")
  df[text_col] = df[text_col].str.replace("á", "a")
  df[text_col] = df[text_col].str.replace("à", "a")
  df[text_col] = df[text_col].str.replace("ñ", "gn")

  return df


diegoToTag = cleaner(diegoToTag, 'concat_text_user_description')

## Functions 

- Classify label_type (service reported)
  - Extracting pound signs (\#)
- Classify Country
  - Matches any state? 
  - has keyword 'edo' or 'estado' in it?

  - Follows any of the common accounts?
- Classify State
  - Match with list of venezuela states
  - We can use a list of venezuelan cities as well 

### Classifying Label Type

#### Hashtags

* \#SinLuz






In [18]:
# 653 hashtags


indices = diegoToTag.concat_text_user_description.str.match('')
diegoToTag[indices]

# indices = diegoToTag.concat_text_user_description.str.match('#')

,concat_text_user_description,label_country,label_state,label_type
